# 6s44

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    if irun % 100 == 0
        print("irun = $irun")
    end
    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1120]
        t2 = t[:, 1121:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

irun = 100irun = 200irun = 300irun = 400irun = 500irun = 600irun = 700irun = 800irun = 900irun = 1000

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

Row,fnames,score,omega,phi,prou,d,struc
,Any,Any,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1006.2,"[0.570888, 0.818896, -0.059122]",176.342,"[73.5321, -31.6233, 272.021]",0.304885,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 200.06 169.46 273.94 |"
2,1/complex.2.pdb,980.059,"[0.0283857, 0.397653, -0.917097]",85.4762,"[222.742, 245.267, 113.242]",0.627472,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 186.91 184.46 252.44 |"
3,1/complex.3.pdb,941.592,"[-0.422624, 0.881435, -0.210856]",71.3854,"[215.281, 165.01, 258.293]",-19.9518,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 209.56 134.15 260.73 |"
4,1/complex.4.pdb,929.164,"[-0.0573369, -0.476652, 0.87722]",83.3622,"[225.041, 244.624, 147.63]",-0.302954,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 251.02 179.36 256.38 |"
5,1/complex.5.pdb,910.584,"[-0.232663, -0.611069, -0.756612]",121.753,"[129.629, -48.0537, -1.05182]",9.98062,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 203.05 136.86 247.30 |"
6,1/complex.6.pdb,909.324,"[0.253445, 0.444156, -0.859355]",76.7009,"[232.41, 222.884, 183.741]",24.9745,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 187.22 189.09 221.21 |"
7,1/complex.7.pdb,903.876,"[0.140658, 0.578577, 0.803408]",124.284,"[162.914, -29.1464, -7.53262]",10.7668,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 199.97 163.93 251.56 |"
8,1/complex.8.pdb,872.951,"[0.0938586, 0.752009, 0.652436]",155.88,"[180.825, -70.6018, 55.3636]",8.29595,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 199.04 155.56 256.46 |"
9,1/complex.9.pdb,866.301,"[0.326708, 0.760451, 0.561227]",161.072,"[100.66, -95.592, 70.9278]",7.24609,"1x2240 TrjArray{Float64, Int64}\n| 210.77 166.19 238.07 | … 196.21 155.45 266.00 |"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)